In [ ]:
!pip install fuzzywuzzy[speedup]

In [ ]:
!pip install fuzzywuzzy[speedup] pandas openpyxl

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz

# Excel 파일에서 데이터를 불러오는 함수
def load_data(file_path):
    return pd.read_excel(file_path, usecols=['식품명', '식품명 동의어', '종류', '종류동의어', '품종', '부위', '부위 동의어'])

# 문자열 정규화 함수
def normalize_string(s):
    return s.lower().strip()

# 유사도 계산 및 상위 5개 유사한 행 찾기 함수
def find_top_similar_rows(input_str, dataframe, top_n=5):
    input_str = normalize_string(input_str)
    similarity_scores = []

    for index, row in dataframe.iterrows():
        row_similarity = sum(fuzz.partial_ratio(input_str, normalize_string(str(item))) for item in row)
        avg_similarity = row_similarity / len(row)
        similarity_scores.append((index, avg_similarity))

    # 상위 5개의 유사한 행 정렬 및 선택
    top_similar_rows = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[:top_n]
    return [(dataframe.iloc[index], score) for index, score in top_similar_rows]

# 파일 경로
file_path = '/content/drive/MyDrive/분석/식재료사전.xlsx'

# 데이터 불러오기
df = load_data(file_path)

# 예시 사용
input_str = "우스터소스또는식초동량으로섞은것"
top_rows = find_top_similar_rows(input_str, df, top_n=5)

# 결과 출력
for row, score in top_rows:
    print("유사도 점수:", score)
    print(row)
    print()


In [ ]:
# 열 몇개만

import pandas as pd
from fuzzywuzzy import fuzz

# Excel 파일에서 데이터를 불러오는 함수
def load_data(file_path):
    return pd.read_excel(file_path, usecols=['식품명', '식품명 동의어', '종류', '종류동의어', '품종', '부위', '부위 동의어'])

# 문자열 정규화 함수
def normalize_string(s):
    return s.lower().strip()

# 유사도 계산 및 상위 5개 유사한 행 찾기 함수
def find_top_similar_rows(input_str, dataframe, top_n=5):
    input_str = normalize_string(input_str)
    similarity_scores = []

    for index, row in dataframe.iterrows():
        row_similarity = 0
        valid_items_count = 0

        for item in row:
            if pd.notna(item):
                row_similarity += fuzz.partial_ratio(input_str, normalize_string(str(item)))
                valid_items_count += 1

        if valid_items_count > 0:  # 유효한 항목이 있을 경우에만 평균 계산
            avg_similarity = row_similarity / valid_items_count
            similarity_scores.append((index, avg_similarity))

    # 상위 5개의 유사한 행 정렬 및 선택
    top_similar_rows = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[:top_n]
    return [(dataframe.iloc[index], score) for index, score in top_similar_rows]

# 파일 경로
file_path_excel = '/content/drive/MyDrive/분석/식재료사전.xlsx'
file_path_text = '/content/drive/MyDrive/분석/만개의레시피에만있는재료.txt'

# 데이터 불러오기
df = load_data(file_path_excel)

# 텍스트 파일에서 한 줄씩 읽기
with open(file_path_text, 'r', encoding='cp949') as file:
    lines = file.readlines()

# 각 줄에 대해 유사도 계산 및 출력
for line in lines:
    top_rows = find_top_similar_rows(line, df, top_n=1)
    print(f"입력: {line.strip()}")
    for row, score in top_rows:
        print("유사도 점수:", score)
        print(row)
    print()


In [ ]:
# 식품명과 동의어에 가중치
import pandas as pd
from fuzzywuzzy import fuzz
import re

# Excel 파일에서 데이터를 불러오는 함수
def load_data(file_path):
    return pd.read_excel(file_path, usecols=['식품명', '식품명 동의어', '종류', '종류동의어', '품종', '부위', '부위 동의어'])

# 문자열 정규화 및 전처리 함수
def preprocess_string(s):
    s = s.lower().strip()  # 소문자 변환 및 공백 제거
    s = re.sub(r'\W+', ' ', s)  # 특수 문자 제거
    return s

# 유사도 계산 및 상위 5개 유사한 행 찾기 함수
def find_top_similar_rows(input_str, dataframe, top_n=5):
    input_str = preprocess_string(input_str)
    similarity_scores = []

    for index, row in dataframe.iterrows():
        row_similarity = 0
        valid_items_count = 0

        for col, item in row.iteritems():
            if pd.notna(item):
                weight = 1.5 if col in ['식품명', '식품명 동의어'] else 1  # '식품명'과 '식품명 동의어'에 높은 가중치 부여
                score = fuzz.token_set_ratio(input_str, preprocess_string(str(item))) * weight
                row_similarity += score
                valid_items_count += weight

        if valid_items_count > 0:  # 유효한 항목이 있을 경우에만 평균 계산
            avg_similarity = row_similarity / valid_items_count
            similarity_scores.append((index, avg_similarity))

    # 상위 5개의 유사한 행 정렬 및 선택
    top_similar_rows = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[:top_n]
    return [(dataframe.iloc[index], score) for index, score in top_similar_rows]

# 파일 경로
file_path_excel = '/content/drive/MyDrive/분석/식재료사전.xlsx'
file_path_text = '/content/drive/MyDrive/분석/만개의레시피에만있는재료.txt'

# 데이터 불러오기
df = load_data(file_path_excel)

# 텍스트 파일에서 한 줄씩 읽기
with open(file_path_text, 'r', encoding='cp949') as file:
    lines = file.readlines()

# 각 줄에 대해 유사도 계산 및 출력
for line in lines:
    top_rows = find_top_similar_rows(line, df, top_n=1)
    print(f"입력: {line.strip()}")
    for row, score in top_rows:
        print("유사도 점수:", score)
        print(row)
    print()


In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
import re

# Excel 파일에서 데이터를 불러오는 함수
def load_data(file_path):
    return pd.read_excel(file_path, usecols=['식품명', '식품명 동의어', '종류', '종류동의어', '품종', '부위', '부위 동의어'])

# 문자열 정규화 및 전처리 함수
def preprocess_string(s):
    s = s.lower().strip()  # 소문자 변환 및 공백 제거
    s = re.sub(r'\W+', ' ', s)  # 특수 문자 제거
    return s

# 유사도 계산 및 상위 5개 유사한 행 찾기 함수
def find_top_similar_rows(input_str, dataframe, top_n=5):
    input_str = preprocess_string(input_str)
    similarity_scores = []

    for index, row in dataframe.iterrows():
        row_similarity = 0
        valid_items_count = 0

        for col, item in row.iteritems():
            if pd.notna(item):
                weight = 1.5 if col in ['식품명', '식품명 동의어'] else 1  # '식품명'과 '식품명 동의어'에 높은 가중치 부여
                score = fuzz.token_set_ratio(input_str, preprocess_string(str(item))) * weight
                row_similarity += score
                valid_items_count += weight

        if valid_items_count > 0:  # 유효한 항목이 있을 경우에만 평균 계산
            avg_similarity = row_similarity / valid_items_count
            similarity_scores.append((index, avg_similarity))

    # 상위 5개의 유사한 행 정렬 및 선택
    top_similar_rows = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[:top_n]
    return [(dataframe.iloc[index], score) for index, score in top_similar_rows]

# 파일 경로
file_path_excel = '/content/drive/MyDrive/분석/식재료사전.xlsx'
file_path_text = '/content/drive/MyDrive/분석/만개의레시피에만있는재료.txt'

# 데이터 불러오기
df = load_data(file_path_excel)

# 텍스트 파일에서 한 줄씩 읽기
with open(file_path_text, 'r', encoding='cp949') as file:
    lines = file.readlines()

# 각 줄에 대해 유사도 계산 및 출력
for line in lines:
    top_rows = find_top_similar_rows(line, df, top_n=1)
    print(f"입력: {line.strip()}")
    for row, score in top_rows:
        print("유사도 점수:", score)
        print(row)
    print()


In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
import re

# Excel 파일에서 데이터를 불러오는 함수
def load_data(file_path):
    return pd.read_excel(file_path, usecols=['식품명', '식품명 동의어', '종류', '종류동의어', '품종', '가공형태', '부위', '부위 동의어'])

# 문자열 정규화 및 전처리 함수
def preprocess_string(s):
    s = s.lower().strip()  # 소문자 변환 및 공백 제거
    s = re.sub(r'\W+', ' ', s)  # 특수 문자 제거
    return s

# 유사도 계산 및 가장 유사한 행 찾기 함수
def find_most_similar_row(input_str, dataframe):
    input_str = preprocess_string(input_str)
    highest_similarity = 0
    most_similar_row = None

    for index, row in dataframe.iterrows():
        # 모든 열에 대해 유사도 계산
        for col in dataframe.columns:
            item_str = preprocess_string(str(row[col]))
            if pd.notna(row[col]):
                similarity = fuzz.token_set_ratio(input_str, item_str)
                if similarity > highest_similarity:
                    highest_similarity = similarity
                    most_similar_row = row

        # '식품명 + 가공형태'와 '식품명 + 부위' 조합의 유사도 계산
        combined_str_processed = preprocess_string(str(row['식품명'])) + " " + preprocess_string(str(row['가공형태']))
        combined_str_part = preprocess_string(str(row['식품명'])) + " " + preprocess_string(str(row['부위']))
        similarity_processed = fuzz.token_set_ratio(input_str, combined_str_processed)
        similarity_part = fuzz.token_set_ratio(input_str, combined_str_part)

        # 가장 높은 유사도를 가진 조합 선택
        best_similarity = max(highest_similarity, similarity_processed, similarity_part)
        if best_similarity > highest_similarity:
            highest_similarity = best_similarity
            most_similar_row = row

    return most_similar_row, highest_similarity

# 파일 경로
file_path_excel = '/content/drive/MyDrive/분석/식재료사전.xlsx'
file_path_text = '/content/drive/MyDrive/분석/만개의레시피에만있는재료.txt'

# 데이터 불러오기
df = load_data(file_path_excel)

# 텍스트 파일에서 한 줄씩 읽기
with open(file_path_text, 'r', encoding='cp949') as file:
    lines = file.readlines()

# 각 줄에 대해 가장 유사한 행 찾기 및 출력
for line in lines:
    most_similar_row, similarity_score = find_most_similar_row(line, df)
    print(f"입력: {line.strip()}")
    print("가장 유사한 행:", most_similar_row)
    print("유사도 점수:", similarity_score)
    print()


In [ ]:
!pip install scikit-learn

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

# Excel 파일에서 데이터를 불러오는 함수
def load_data(file_path):
    return pd.read_excel(file_path, usecols=['식품명', '식품명 동의어', '종류', '종류동의어', '품종', '가공형태', '부위', '부위 동의어'])

# 문자열 정규화 및 전처리 함수
def preprocess_string(s):
    s = s.lower().strip()  # 소문자 변환 및 공백 제거
    s = re.sub(r'\W+', ' ', s)  # 특수 문자 제거
    return s

# 텍스트를 하나의 문자열로 결합하는 함수
def combine_text(row):
    return ' '.join(preprocess_string(str(row[col])) for col in row.index if pd.notna(row[col]))

# TF-IDF 유사도 계산 함수
def calculate_similarity(input_str, dataframe):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(dataframe['combined_text'].tolist() + [input_str])
    cosine_sim = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
    return cosine_sim[0]

# 파일 경로
file_path_excel = '/content/drive/MyDrive/분석/식재료사전.xlsx'
file_path_text = '/content/drive/MyDrive/분석/만개의레시피에만있는재료.txt'

# 데이터 불러오기
df = load_data(file_path_excel)
df['combined_text'] = df.apply(combine_text, axis=1)

# 텍스트 파일에서 한 줄씩 읽기
with open(file_path_text, 'r', encoding='cp949') as file:
    lines = file.readlines()

# 각 줄에 대해 유사도 계산 및 출력
for line in lines:
    input_str = preprocess_string(line.strip())
    similarities = calculate_similarity(input_str, df)
    most_similar_idx = similarities.argmax()
    most_similar_row = df.iloc[most_similar_idx]
    similarity_score = similarities[most_similar_idx]

    print(f"입력: {input_str}")
    print("가장 유사한 행:", most_similar_row['combined_text'])
    print("유사도 점수:", similarity_score)
    print()


In [ ]:
# 이 코드 써야됨

import pandas as pd
from fuzzywuzzy import fuzz
import re

# Excel 파일에서 데이터를 불러오는 함수
def load_data(file_path):
    return pd.read_excel(file_path, usecols=['식품명', '식품명 동의어', '종류', '종류동의어', '품종', '가공형태', '부위', '부위 동의어'])

# 문자열 정규화 및 전처리 함수
def preprocess_string(s):
    s = s.lower().strip()  # 소문자 변환 및 공백 제거
    s = re.sub(r'\W+', ' ', s)  # 특수 문자 제거
    return s

# 유사도 계산 및 가장 유사한 행 찾기 함수
def find_most_similar_row(input_str, dataframe, name_weight=2.0, combo_weight=1.5):
    input_str = preprocess_string(input_str)
    highest_similarity = 0
    most_similar_row = None

    for index, row in dataframe.iterrows():
        # '식품명' 열에 높은 가중치 적용
        name_similarity = fuzz.token_set_ratio(input_str, preprocess_string(str(row['식품명'])))
        if name_similarity > highest_similarity:
            highest_similarity = name_similarity * name_weight
            most_similar_row = row

        # 다른 열에 대해 유사도 계산
        for col in ['식품명 동의어', '종류', '종류동의어', '품종', '가공형태', '부위', '부위 동의어']:
            if pd.notna(row[col]):
                col_similarity = fuzz.token_set_ratio(input_str, preprocess_string(str(row[col])))
                if col_similarity > highest_similarity:
                    highest_similarity = col_similarity
                    most_similar_row = row

        # 조합에 대한 유사도 계산
        for col1, col2 in [('식품명', '부위'), ('식품명', '가공형태')]:
            combined_str = preprocess_string(str(row[col1])) + " " + preprocess_string(str(row[col2]))
            combined_similarity = fuzz.token_set_ratio(input_str, combined_str) * combo_weight
            if combined_similarity > highest_similarity:
                highest_similarity = combined_similarity
                most_similar_row = row

    return most_similar_row, highest_similarity / max(name_weight, combo_weight)

# 파일 경로
file_path_excel = '/content/drive/MyDrive/분석/식재료사전.xlsx'
file_path_text = '/content/drive/MyDrive/분석/만개의레시피에만있는재료.txt'

# 데이터 불러오기
df = load_data(file_path_excel)

# 텍스트 파일에서 한 줄씩 읽기
with open(file_path_text, 'r', encoding='cp949') as file:
    lines = file.readlines()

# 각 줄에 대해 가장 유사한 행 찾기 및 출력
for line in lines:
    most_similar_row, similarity_score = find_most_similar_row(line, df)
    print(f"입력: {line.strip()}")
    print("가장 유사한 행:", most_similar_row)
    print("유사도 점수:", similarity_score)
    print()


In [ ]:
# 이 코드 써야됨

import pandas as pd
from fuzzywuzzy import fuzz
import re

# Excel 파일에서 데이터를 불러오는 함수
def load_data(file_path):
    return pd.read_excel(file_path, usecols=['식품명', '식품명 동의어', '종류', '종류동의어', '품종', '부위', '부위 동의어'])

# 문자열 정규화 및 전처리 함수
def preprocess_string(s):
    s = s.lower().strip()  # 소문자 변환 및 공백 제거
    s = re.sub(r'\W+', ' ', s)  # 특수 문자 제거
    return s

# 유사도 계산 및 가장 유사한 행 찾기 함수
def find_most_similar_row(input_str, dataframe, name_weight=2.0, combo_weight=1.5):
    input_str = preprocess_string(input_str)
    highest_similarity = 0
    most_similar_row = None

    for index, row in dataframe.iterrows():
        # '식품명' 열에 높은 가중치 적용
        name_similarity = fuzz.token_set_ratio(input_str, preprocess_string(str(row['식품명'])))
        if name_similarity > highest_similarity:
            highest_similarity = name_similarity * name_weight
            most_similar_row = row

        # 다른 열에 대해 유사도 계산
        for col in ['식품명 동의어', '종류', '종류동의어', '품종', '부위', '부위 동의어']:
            if pd.notna(row[col]):
                col_similarity = fuzz.token_set_ratio(input_str, preprocess_string(str(row[col])))
                if col_similarity > highest_similarity:
                    highest_similarity = col_similarity
                    most_similar_row = row

        # 조합에 대한 유사도 계산
        for col1, col2 in [('식품명', '부위')]:
            combined_str = preprocess_string(str(row[col1])) + " " + preprocess_string(str(row[col2]))
            combined_similarity = fuzz.token_set_ratio(input_str, combined_str) * combo_weight
            if combined_similarity > highest_similarity:
                highest_similarity = combined_similarity
                most_similar_row = row

    return most_similar_row, highest_similarity / max(name_weight, combo_weight)

# 파일 경로
file_path_excel = '/content/drive/MyDrive/분석/식재료사전.xlsx'
file_path_text = '/content/drive/MyDrive/분석/만개의레시피에만있는재료.txt'

# 데이터 불러오기
df = load_data(file_path_excel)

# 텍스트 파일에서 한 줄씩 읽기
with open(file_path_text, 'r', encoding='cp949') as file:
    lines = file.readlines()

# 각 줄에 대해 가장 유사한 행 찾기 및 출력
for line in lines:
    most_similar_row, similarity_score = find_most_similar_row(line, df)
    print(f"입력: {line.strip()}")
    print("가장 유사한 행:", most_similar_row)
    print("유사도 점수:", similarity_score)
    print()


In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
import re

# Excel 파일에서 데이터를 불러오는 함수
def load_data(file_path):
    return pd.read_excel(file_path, usecols=['식품명', '식품명 동의어', '종류', '종류동의어', '품종', '부위', '부위 동의어'])

# 문자열 정규화 및 전처리 함수
def preprocess_string(s):
    s = s.lower().strip()  # 소문자 변환 및 공백 제거
    s = re.sub(r'\W+', ' ', s)  # 특수 문자 제거
    return s

# 유사도 계산 및 가장 유사한 행 찾기 함수
def find_most_similar_row(input_str, dataframe, name_weight=2.0, combo_weight=1.5):
    input_str = preprocess_string(input_str)
    highest_similarity = 0
    most_similar_row = None

    for index, row in dataframe.iterrows():
        # '식품명' 열에 높은 가중치 적용
        if pd.notna(row['식품명']):
            name_similarity = fuzz.token_set_ratio(input_str, preprocess_string(str(row['식품명'])))
            if name_similarity > highest_similarity:
                highest_similarity = name_similarity * name_weight
                most_similar_row = row

        # 다른 열에 대해 유사도 계산
        for col in ['식품명 동의어', '종류', '종류동의어', '품종', '부위', '부위 동의어']:
            if pd.notna(row[col]):
                col_similarity = fuzz.token_set_ratio(input_str, preprocess_string(str(row[col])))
                if col_similarity > highest_similarity:
                    highest_similarity = col_similarity
                    most_similar_row = row

        # 조합에 대한 유사도 계산
        for col1, col2 in [('식품명', '부위')]:
            if pd.notna(row[col1]) and pd.notna(row[col2]):
                combined_str = preprocess_string(str(row[col1])) + " " + preprocess_string(str(row[col2]))
                combined_similarity = fuzz.token_set_ratio(input_str, combined_str) * combo_weight
                if combined_similarity > highest_similarity:
                    highest_similarity = combined_similarity
                    most_similar_row = row

    return most_similar_row, highest_similarity / max(name_weight, combo_weight)

# 유사도 계산 함수
def calculate_string_ratios(input_str, comparison_str):
    input_str = preprocess_string(input_str)
    comparison_str = preprocess_string(comparison_str)

    # 재료사전 '식품명'에 입력 문자열이 포함되는 비율
    input_in_comparison = sum(char in comparison_str for char in input_str) / len(input_str) if input_str else 0

    # 입력 문자열에 재료사전 '식품명'이 포함되는 비율
    comparison_in_input = sum(char in input_str for char in comparison_str) / len(comparison_str) if comparison_str else 0

    # 글자 단위 겹치는 비율
    intersection = set(input_str) & set(comparison_str)
    union = set(input_str) | set(comparison_str)
    overlap_ratio = len(intersection) / len(union) if union else 0

    return input_in_comparison, comparison_in_input, overlap_ratio

# 파일 경로
file_path_excel = '/content/drive/MyDrive/분석/식재료사전.xlsx'
file_path_text = '/content/drive/MyDrive/분석/만개의레시피에만있는재료.txt'

# 결과를 저장할 리스트
result_list = []

# 데이터 불러오기
df = load_data(file_path_excel)

# 텍스트 파일에서 한 줄씩 읽기
with open(file_path_text, 'r', encoding='cp949') as file:
    lines = file.readlines()

# 각 줄에 대해 가장 유사한 행 찾기 및 결과 저장
for line in lines:
    most_similar_row, similarity_score = find_most_similar_row(line, df)
    if most_similar_row is not None:
        input_str = line.strip()
        name_str = str(most_similar_row['식품명'])
        ratios = calculate_string_ratios(input_str, name_str)

        # 결과를 딕셔너리로 저장
        result_dict = {
            "입력 문자열": input_str,
            "재료사전 식품명": name_str,
            "입력 문자열에 포함 비율": ratios[0],
            "재료사전 식품명에 포함 비율": ratios[1],
            "글자 단위 겹치는 비율": ratios[2]
        }
        result_list.append(result_dict)

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(result_list)

# 결과를 파일로 저장 (CSV 형식)
result_file_path = '/content/drive/MyDrive/분석/비율결과.csv'
result_df.to_csv(result_file_path, index=False)

# 결과를 출력
print(result_df)

# 파일 경로 출력
print(f"결과가 다음 경로에 저장되었습니다: {result_file_path}")


In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
import re

# 주어진 함수들
def load_data(file_path):
    return pd.read_excel(file_path, usecols=['식품명', '식품명 동의어', '종류', '종류동의어', '품종', '부위', '부위 동의어'])

def preprocess_string(s):
    s = s.lower().strip()
    s = re.sub(r'\W+', ' ', s)
    return s

def find_most_similar_row(input_str, dataframe, name_weight=2.0, combo_weight=1.5):
    input_str = preprocess_string(input_str)
    highest_similarity = 0
    most_similar_row = None

    for index, row in dataframe.iterrows():
        name_similarity = fuzz.token_set_ratio(input_str, preprocess_string(str(row['식품명'])))
        if name_similarity > highest_similarity:
            highest_similarity = name_similarity * name_weight
            most_similar_row = row

        for col in ['식품명 동의어', '종류', '종류동의어', '품종', '부위', '부위 동의어']:
            if pd.notna(row[col]):
                col_similarity = fuzz.token_set_ratio(input_str, preprocess_string(str(row[col])))
                if col_similarity > highest_similarity:
                    highest_similarity = col_similarity
                    most_similar_row = row

        for col1, col2 in [('식품명', '부위')]:
            combined_str = preprocess_string(str(row[col1])) + " " + preprocess_string(str(row[col2]))
            combined_similarity = fuzz.token_set_ratio(input_str, combined_str) * combo_weight
            if combined_similarity > highest_similarity:
                highest_similarity = combined_similarity
                most_similar_row = row

    return most_similar_row, highest_similarity / max(name_weight, combo_weight)

# 파일 경로
file_path_excel = '/content/drive/MyDrive/분석/식재료사전.xlsx'
file_path_text = '/content/drive/MyDrive/분석/만개의레시피에만있는재료.txt'

# 데이터 불러오기
df = load_data(file_path_excel)

# 결과를 저장할 리스트
results = []

# 텍스트 파일에서 한 줄씩 읽기 및 결과 처리
with open(file_path_text, 'r', encoding='cp949') as file:
    lines = file.readlines()

    for line in lines:
        most_similar_row, similarity_score = find_most_similar_row(line, df)
        result = {
            "입력": line.strip(),
            "가장 유사한 행": most_similar_row.to_dict() if most_similar_row is not None else 'None',
            "유사도 점수": similarity_score
        }
        results.append(result)

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 결과를 텍스트 파일로 저장
output_file_path = '/content/drive/MyDrive/분석/비율결과.txt'
with open(output_file_path, 'w', encoding='utf-8') as file:
    for result in results:
        file.write(str(result) + '\n')


In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
import re

# Excel 파일에서 데이터를 불러오는 함수
def load_data(file_path):
    return pd.read_excel(file_path, usecols=['식품명', '식품명 동의어', '종류', '종류동의어', '품종', '부위', '부위 동의어'])

# 문자열 정규화 및 전처리 함수
def preprocess_string(s):
    s = s.lower().strip()  # 소문자 변환 및 공백 제거
    s = re.sub(r'\W+', ' ', s)  # 특수 문자 제거
    return s

# 유사도 계산 및 가장 유사한 행 찾기 함수
def find_most_similar_row(input_str, dataframe, name_weight=2.0, combo_weight=1.5):
    input_str = preprocess_string(input_str)
    highest_similarity = 0
    most_similar_row = None

    for index, row in dataframe.iterrows():
        name_similarity = fuzz.token_set_ratio(input_str, preprocess_string(str(row['식품명'])))
        if name_similarity > highest_similarity:
            highest_similarity = name_similarity * name_weight
            most_similar_row = row

        for col in ['식품명 동의어', '종류', '종류동의어', '품종', '부위', '부위 동의어']:
            if pd.notna(row[col]):
                col_similarity = fuzz.token_set_ratio(input_str, preprocess_string(str(row[col])))
                if col_similarity > highest_similarity:
                    highest_similarity = col_similarity
                    most_similar_row = row

        for col1, col2 in [('식품명', '부위')]:
            combined_str = preprocess_string(str(row[col1])) + " " + preprocess_string(str(row[col2]))
            combined_similarity = fuzz.token_set_ratio(input_str, combined_str) * combo_weight
            if combined_similarity > highest_similarity:
                highest_similarity = combined_similarity
                most_similar_row = row

    return most_similar_row, highest_similarity / max(name_weight, combo_weight)

# 파일 경로
file_path_excel = '/content/drive/MyDrive/분석/식재료사전.xlsx'
file_path_text = '/content/drive/MyDrive/분석/만개의레시피에만있는재료.txt'

# 데이터 불러오기
df = load_data(file_path_excel)

# 결과를 저장할 리스트
results = []

# 텍스트 파일에서 한 줄씩 읽기
with open(file_path_text, 'r', encoding='cp949') as file:
    lines = file.readlines()

    for line in lines:
        most_similar_row, similarity_score = find_most_similar_row(line, df)
        result = {
            "입력": line.strip(),
            "가장 유사한 행": most_similar_row.to_dict() if most_similar_row is not None else 'None',
            "유사도 점수": similarity_score
        }
        results.append(result)

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 결과를 Excel 파일로 저장
output_file_path = '유사도_결과.xlsx'  # 로컬 경로로 변경 필요
results_df.to_excel(output_file_path, index=False)
